In [160]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.layers import Input, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score

import tensorflow as tf
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.layers import concatenate

import warnings
warnings.filterwarnings("ignore")

In [161]:
df = pd.read_csv('ASAP Dataset/Preprocessed_df.csv')

In [162]:
df.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   rater1_domain1  rater2_domain1  rater3_domain1  domain1_score  \
0               4               4             NaN              8   
1               5               4             NaN              9   
2               4               3             NaN              7   
3               5               5             NaN             10   
4               4               4             NaN              8   

   rater1_domain2  rater2_domain2  domain2_score  ...  word_len  chars_len  \
0             NaN             NaN            NaN  ...       386       1875   
1             NaN             NaN            NaN  ...       464       2288   
2             NaN             NaN            NaN  ...       313       1541   
3             NaN             NaN            NaN  ...       611       3165   
4             NaN             NaN            NaN  ...       517       2569   

   avg_word_length  avg_sentence_length  \
0         3.984456                  1.0   
1         4.030172                  1.0   
2         4.035144                  1.0   
3         4.328969                  1.0   
4         4.071567                  1.0   

                                          pos_ratios  num_sentences  \
0  {'NNP': 0.031088082901554404, 'JJ': 0.05181347...             16   
1  {'NNP': 0.03879310344827586, ',': 0.0258620689...             20   
2  {'NNP': 0.04153354632587859, ',': 0.0287539936...             14   
3  {'NNP': 0.11620294599018004, ',': 0.0212765957...             27   
4  {'NNP': 0.017408123791102514, ',': 0.025145067...             30   

   num_paragraphs  sentiment_polariy  sentiment_subjectivity  \
0               1           0.310471                0.385613   
1               1           0.274000                0.613167   
2               1           0.340393                0.498657   
3               1           0.266828                0.441795   
4               1           0.199684                0.485814   

                                   preprocessed_text  
0  dear local newspaper think effect computer peo...  
1  dear believe using computer benefit u many way...  
2  dear people use computer everyone agrees benef...  
3  dear local newspaper found many expert say com...  
4  dear know computer positive effect people comp...  

[5 rows x 38 columns]

In [163]:
df = df.dropna(axis = 1, how = 'any')

In [164]:
drop_columns = ['essay_id', 'pos_ratios', 'essay', 'rater1_domain1', 'rater2_domain1']
df.drop(drop_columns, axis = 1, inplace = True)

In [165]:
def calculate_precision(y_true, y_pred, average='macro'):
    precision = precision_score(y_true, y_pred, average=average)
    return precision

def calculate_recall(y_true, y_pred, average='macro'):
    recall = recall_score(y_true, y_pred, average=average)
    return recall

def calculate_f1_score(y_true, y_pred, average='macro'):
    f1 = f1_score(y_true, y_pred, average=average)
    return f1

def calculate_cohen_kappa_score(y_true, y_pred):
    kappa_score = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
    return kappa_score

def calculate_accuracy(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

def print_metrics_function(y_actual, y_predictions):
    
    accuracy = calculate_accuracy(y_actual, y_predictions)
    precision = calculate_precision(y_actual, y_predictions)
    recall = calculate_recall(y_actual, y_predictions)
    kappa_score = calculate_cohen_kappa_score(y_actual, y_predictions)

    return accuracy, precision, recall, f1, kappa_score

In [168]:
def dataset_preparation(data, target = 'domain1_score'):
    
    X = data.drop([target], axis = 1)
    y = data[target]
    
    return X, y

### Model with Metrics (Essay Set - 1)

In [169]:
df_essay_set = df[df.essay_set == 1]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [170]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (1426, 10)
The shape of the input test data: (357, 10)
---------------------------------------------------
The shape of the output train data: (1426, 13)
The shape of the output test data: (357, 13)


In [171]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [172]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
45/45 [==============================] - 2s 38ms/step - loss: 1.9530 - accuracy: 0.3759 - val_loss: 1.7729 - val_accuracy: 0.3641
Epoch 2/10
45/45 [==============================] - 1s 19ms/step - loss: 1.6472 - accuracy: 0.4032 - val_loss: 1.7229 - val_accuracy: 0.3697
Epoch 3/10
45/45 [==============================] - 1s 17ms/step - loss: 1.4578 - accuracy: 0.4691 - val_loss: 1.6887 - val_accuracy: 0.3922
Epoch 4/10
45/45 [==============================] - 1s 17ms/step - loss: 1.1898 - accuracy: 0.5743 - val_loss: 1.6686 - val_accuracy: 0.3866
Epoch 5/10
45/45 [==============================] - 1s 18ms/step - loss: 0.9309 - accuracy: 0.6725 - val_loss: 1.6663 - val_accuracy: 0.3669
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.7672 - accuracy: 0.7363 - val_loss: 1.7484 - val_accuracy: 0.3754
Epoch 7/10
45/45 [==============================] - 1s 17ms/step - loss: 0.5784 - accuracy: 0.8079 - val_loss: 1.8298 - val_accuracy: 0.3725
Epoch 8/10
45

In [173]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.3445378151260504
Precision: 0.1839159439447185
Recall: 0.167596345187071
F1-Score: 0.150305207126839
Cohen Kappa Score: 0.34645857179608586


(0.3445378151260504,
 0.1839159439447185,
 0.167596345187071,
 0.150305207126839,
 0.34645857179608586)

### Model with Metrics (Essay Set - 2)

In [174]:
df_essay_set = df[df.essay_set == 2]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [175]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (1440, 10)
The shape of the input test data: (360, 10)
---------------------------------------------------
The shape of the output train data: (1440, 7)
The shape of the output test data: (360, 7)


In [176]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [177]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
45/45 [==============================] - 1s 25ms/step - loss: 1.3368 - accuracy: 0.4333 - val_loss: 1.1666 - val_accuracy: 0.4639
Epoch 2/10
45/45 [==============================] - 1s 16ms/step - loss: 1.0737 - accuracy: 0.5639 - val_loss: 1.1170 - val_accuracy: 0.5111
Epoch 3/10
45/45 [==============================] - 1s 16ms/step - loss: 0.9078 - accuracy: 0.6715 - val_loss: 1.1259 - val_accuracy: 0.5028
Epoch 4/10
45/45 [==============================] - 1s 17ms/step - loss: 0.6972 - accuracy: 0.7500 - val_loss: 1.1253 - val_accuracy: 0.5472
Epoch 5/10
45/45 [==============================] - 1s 15ms/step - loss: 0.5006 - accuracy: 0.8222 - val_loss: 1.3227 - val_accuracy: 0.5194
Epoch 6/10
45/45 [==============================] - 1s 15ms/step - loss: 0.4324 - accuracy: 0.8542 - val_loss: 1.2892 - val_accuracy: 0.5306
Epoch 7/10
45/45 [==============================] - 1s 16ms/step - loss: 0.3442 - accuracy: 0.8826 - val_loss: 1.3360 - val_accuracy: 0.5500
Epoch 8/10
45

In [178]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.5055555555555555
Precision: 0.2729098488390213
Recall: 0.2901935812531839
F1-Score: 0.28066848634424596
Cohen Kappa Score: 0.3961397058823529


(0.5055555555555555,
 0.2729098488390213,
 0.2901935812531839,
 0.28066848634424596,
 0.3961397058823529)

### Model with Metrics (Essay Set - 4)

In [206]:
df_essay_set = df[df.essay_set == 4]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [207]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (1416, 10)
The shape of the input test data: (354, 10)
---------------------------------------------------
The shape of the output train data: (1416, 4)
The shape of the output test data: (354, 4)


In [208]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [209]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=2, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
45/45 [==============================] - 1s 33ms/step - loss: 1.2754 - accuracy: 0.4110 - val_loss: 1.1037 - val_accuracy: 0.5169
Epoch 2/10
45/45 [==============================] - 0s 11ms/step - loss: 1.0183 - accuracy: 0.5360 - val_loss: 0.9445 - val_accuracy: 0.5593
Epoch 3/10
45/45 [==============================] - 1s 11ms/step - loss: 0.8771 - accuracy: 0.6243 - val_loss: 0.9334 - val_accuracy: 0.5537
Epoch 4/10
45/45 [==============================] - 1s 11ms/step - loss: 0.7492 - accuracy: 0.6808 - val_loss: 0.9408 - val_accuracy: 0.5706
Epoch 5/10
45/45 [==============================] - 0s 11ms/step - loss: 0.7033 - accuracy: 0.6935 - val_loss: 1.0079 - val_accuracy: 0.5678
Epoch 6/10
45/45 [==============================] - 0s 11ms/step - loss: 0.6367 - accuracy: 0.7288 - val_loss: 1.0080 - val_accuracy: 0.5678
Epoch 7/10
45/45 [==============================] - 1s 12ms/step - loss: 0.5709 - accuracy: 0.7556 - val_loss: 1.1418 - val_accuracy: 0.5621
Epoch 8/10
45

In [210]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.5932203389830508
Precision: 0.6125039361979661
Recall: 0.533907416091646
F1-Score: 0.548260722338866
Cohen Kappa Score: 0.6454184111505644


(0.5932203389830508,
 0.6125039361979661,
 0.533907416091646,
 0.548260722338866,
 0.6454184111505644)

### Model with Metrics (Essay Set - 5)

In [211]:
df_essay_set = df[df.essay_set == 5]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [212]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (1444, 10)
The shape of the input test data: (361, 10)
---------------------------------------------------
The shape of the output train data: (1444, 5)
The shape of the output test data: (361, 5)


In [213]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [214]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
46/46 [==============================] - 1s 23ms/step - loss: 1.3673 - accuracy: 0.3906 - val_loss: 1.1768 - val_accuracy: 0.4681
Epoch 2/10
46/46 [==============================] - 1s 12ms/step - loss: 1.1273 - accuracy: 0.4612 - val_loss: 1.0572 - val_accuracy: 0.5291
Epoch 3/10
46/46 [==============================] - 1s 11ms/step - loss: 0.9394 - accuracy: 0.6046 - val_loss: 0.9982 - val_accuracy: 0.5429
Epoch 4/10
46/46 [==============================] - 1s 11ms/step - loss: 0.7826 - accuracy: 0.6773 - val_loss: 1.0462 - val_accuracy: 0.5679
Epoch 5/10
46/46 [==============================] - 1s 12ms/step - loss: 0.6440 - accuracy: 0.7292 - val_loss: 1.0593 - val_accuracy: 0.5457
Epoch 6/10
46/46 [==============================] - 1s 12ms/step - loss: 0.5217 - accuracy: 0.7978 - val_loss: 1.1942 - val_accuracy: 0.5097
Epoch 7/10
46/46 [==============================] - 1s 11ms/step - loss: 0.4591 - accuracy: 0.8151 - val_loss: 1.2305 - val_accuracy: 0.5402
Epoch 8/10
46

In [215]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.4930747922437673
Precision: 0.3903352650101876
Recall: 0.3668858418891784
F1-Score: 0.37316175994098716
Cohen Kappa Score: 0.5901031608163583


(0.4930747922437673,
 0.3903352650101876,
 0.3668858418891784,
 0.37316175994098716,
 0.5901031608163583)

### Model with Metrics (Essay Set - 6)

In [216]:
df_essay_set = df[df.essay_set == 6]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [217]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (1440, 10)
The shape of the input test data: (360, 10)
---------------------------------------------------
The shape of the output train data: (1440, 5)
The shape of the output test data: (360, 5)


In [218]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [219]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
45/45 [==============================] - 1s 18ms/step - loss: 1.4035 - accuracy: 0.3896 - val_loss: 1.1255 - val_accuracy: 0.4972
Epoch 2/10
45/45 [==============================] - 0s 11ms/step - loss: 1.1798 - accuracy: 0.4750 - val_loss: 1.0659 - val_accuracy: 0.5250
Epoch 3/10
45/45 [==============================] - 0s 11ms/step - loss: 1.0425 - accuracy: 0.5333 - val_loss: 0.9829 - val_accuracy: 0.5472
Epoch 4/10
45/45 [==============================] - 0s 11ms/step - loss: 0.8662 - accuracy: 0.6354 - val_loss: 0.9794 - val_accuracy: 0.5361
Epoch 5/10
45/45 [==============================] - 0s 11ms/step - loss: 0.7601 - accuracy: 0.6729 - val_loss: 0.9620 - val_accuracy: 0.5611
Epoch 6/10
45/45 [==============================] - 0s 11ms/step - loss: 0.6353 - accuracy: 0.7424 - val_loss: 1.0501 - val_accuracy: 0.5667
Epoch 7/10
45/45 [==============================] - 1s 11ms/step - loss: 0.5144 - accuracy: 0.7889 - val_loss: 1.0203 - val_accuracy: 0.5556
Epoch 8/10
45

In [220]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.55
Precision: 0.4080289876858707
Recall: 0.39879325876435706
F1-Score: 0.4019728936534427
Cohen Kappa Score: 0.5869056897895557


(0.55,
 0.4080289876858707,
 0.39879325876435706,
 0.4019728936534427,
 0.5869056897895557)

### Model with Metrics (Essay Set - 8)

In [225]:
df_essay_set = df[df.essay_set == 8]
X, y = dataset_preparation(df_essay_set)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [226]:
print("The shape of the input train data: {}".format(X_train.shape))
print("The shape of the input test data: {}".format(X_test.shape))
print("---------------------------------------------------")
print("The shape of the output train data: {}".format(y_train.shape))
print("The shape of the output test data: {}".format(y_test.shape))

The shape of the input train data: (578, 10)
The shape of the input test data: (145, 10)
---------------------------------------------------
The shape of the output train data: (578, 61)
The shape of the output test data: (145, 61)


In [227]:
X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values

In [228]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_text'])
sequences_train = tokenizer.texts_to_sequences(X_train['preprocessed_text'])
sequences_test = tokenizer.texts_to_sequences(X_test['preprocessed_text'])

# Pad the sequences to a fixed length
max_length = 750
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_length)

X_train_additional_features = X_train[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')
X_test_additional_features = X_test[['word_len', 'chars_len', 'avg_word_length', 'avg_sentence_length',
       'num_sentences', 'num_paragraphs', 'sentiment_polariy',
       'sentiment_subjectivity']].values.astype('int32')

# Define the model architecture
text_input = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length)(text_input)
conv_layer = Conv1D(filters=10, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.2)(pooling_layer)
dense_layer1 = Dense(units=16, activation='relu')(dropout_layer)
output_layer = Dense(units=y_train.shape[1], activation='softmax')(dense_layer1)
model = Model(inputs = text_input, outputs = output_layer)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

padded_sequences_train_concat = concatenate([padded_sequences_train, X_train_additional_features])
padded_sequences_test_concat = concatenate([padded_sequences_test, X_test_additional_features])

# Train the model
model.fit(padded_sequences_train_concat, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_test_concat, y_test))


Epoch 1/10
19/19 [==============================] - 1s 39ms/step - loss: 3.8664 - accuracy: 0.0640 - val_loss: 3.4074 - val_accuracy: 0.0483
Epoch 2/10
19/19 [==============================] - 0s 21ms/step - loss: 3.0212 - accuracy: 0.2145 - val_loss: 3.0246 - val_accuracy: 0.2000
Epoch 3/10
19/19 [==============================] - 0s 17ms/step - loss: 2.7470 - accuracy: 0.2318 - val_loss: 3.0007 - val_accuracy: 0.2000
Epoch 4/10
19/19 [==============================] - 0s 14ms/step - loss: 2.5699 - accuracy: 0.2612 - val_loss: 2.9928 - val_accuracy: 0.2000
Epoch 5/10
19/19 [==============================] - 0s 17ms/step - loss: 2.3661 - accuracy: 0.3080 - val_loss: 3.0283 - val_accuracy: 0.1448
Epoch 6/10
19/19 [==============================] - 0s 16ms/step - loss: 2.1821 - accuracy: 0.3668 - val_loss: 3.0697 - val_accuracy: 0.1103
Epoch 7/10
19/19 [==============================] - 0s 15ms/step - loss: 1.9499 - accuracy: 0.4239 - val_loss: 3.1275 - val_accuracy: 0.1310
Epoch 8/10
19

In [229]:
y_predictions = model.predict(padded_sequences_test_concat)
y_predictions = np.argmax(y_predictions, axis = 1)
print_metrics_function(np.argmax(y_test, axis = 1), y_predictions)

Accuracy: 0.14482758620689656
Precision: 0.019693877551020405
Recall: 0.04356921813818366
F1-Score: 0.025971682336431417
Cohen Kappa Score: -0.09739861043787368


(0.14482758620689656,
 0.019693877551020405,
 0.04356921813818366,
 0.025971682336431417,
 -0.09739861043787368)